<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание" data-toc-modified-id="Задание-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание</a></span></li><li><span><a href="#Задание-*" data-toc-modified-id="Задание-*-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задание *</a></span><ul class="toc-item"><li><span><a href="#Список-объектов" data-toc-modified-id="Список-объектов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Список объектов</a></span></li></ul></li><li><span><a href="#Данные-по-определенному-объекту" data-toc-modified-id="Данные-по-определенному-объекту-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Данные по определенному объекту</a></span></li></ul></div>

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests
import pandas as pd
from tqdm.auto import tqdm
import time
import sqlite3
import plotly.express as px


D:\anaconda\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Список объектов

In [ ]:
#запрос не падает с лимитом 5000, в count считаю кол-во сдвигов offset
limit_ = 5000
url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset?limit?&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()
total_obj = objects_data.get('data').get('total')
count = objects_data.get('data').get('total')//5000
print("Кол-во объектов:", total_obj)

In [ ]:
# Собираю все ID объектов по 5000 и сдвигая offset на 5000
url_ = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={}&limit={}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'

objId = []
for i in range(0,count+1):
    res_ = requests.get(url_.format(limit_*i,limit_))
    res_objId = res_.json()
    for el in res_objId.get('data').get('list'):
        objId.append(el.get('objId'))

# Данные по определенному объекту

In [ ]:
# По каждому ID объекта собираю полное инфо объекта

url_obj = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{}'
objects_data = {'OBJ':[] }
for i in tqdm(objId):
    res = requests.get(url_obj.format(i))
    objects_list = res.json()
    objects_data['OBJ'].append(objects_list['data'])


In [ ]:
df = pd.json_normalize(objects_data.get('OBJ'))
df

,id,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,...,metro.time,metro.isWalk,metro.colors,greenAreaIndexValue,airQualityIndexValue,complexShortNm,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13399,28141,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,Другое,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13398,28141,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,Другое,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13383,28105,72,г Тюмень,4,20,68,2022-06-30,Другое,По желанию заказчика,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13397,28141,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,Другое,Без отделки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13391,28101,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-06-30,Другое,По желанию заказчика,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10074,28071,99284,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",26,26,288,2024-06-30,Другое,Под ключ,...,20.716667,False,[#943E90],NaN,8.0,NaN,NaN,NaN,NaN,NaN
10075,28073,96446,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",26,26,312,2024-06-30,Другое,Под ключ,...,23.283333,True,[#943E90],NaN,8.0,NaN,NaN,NaN,NaN,NaN
10076,44447,475093,76,"г Ярославль, Фрунзенский район, территория в р...",6,6,127,2022-12-31,Кирпич,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,28000,91796,62,"г Рязань, переулок Парковый, д. 1",20,25,422,2022-06-30,Монолит-кирпич,Под чистовую,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('domrf.csv', index= False)

In [ ]:
df.to_excel('domrf.xlsx', index=False)

In [ ]:
df.to_pickle("domrf.pkl")

In [ ]:
conn = sqlite3.connect('test_database')
df.to_sql('domrf', conn, if_exists='replace', index = False)
conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('test_database')

pd.read_sql('''
    SELECT *
    FROM domrf
    where region = 63
    LIMIT 5
''', conn)

,id,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,...,metro.time,metro.isWalk,metro.colors,greenAreaIndexValue,airQualityIndexValue,complexShortNm,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,18136,43084,63,"г Самара, улица Советской Армии, д. 0, корпус ...",27,27,360,2022-12-31,Монолит-кирпич,Без отделки,...,11.683333,1.0,['#CD0505'],None,None,None,None,None,None,None
1,18135,42847,63,"г Самара, район Кировский, пр-кт Кирова, д. 399б",25,25,414,2022-06-30,Монолит-кирпич,Без отделки,...,NaN,NaN,None,None,None,Орбита,None,None,None,None
2,4024,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,16,2022-12-31,Кирпич,Без отделки,...,NaN,NaN,None,None,None,Акварель,None,None,None,None
3,4025,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,Без отделки,...,NaN,NaN,None,None,None,Акварель,None,None,None,None
4,4023,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,48,2022-12-31,Кирпич,Без отделки,...,NaN,NaN,None,None,None,Акварель,None,None,None,None


In [2]:
df_dom = pd.read_excel('domrf.xlsx')

In [3]:
df_dom.shape

(10079, 108)

In [3]:
#Проверил сколько пропусков и оставил сттобцы где меньше 60% пропусков
df_dom.isnull().sum()

id                                              0
pdId                                            0
region                                          0
address                                        26
floorMin                                        0
                                             ... 
complexShortNm                               9153
developer.bankruptStage.bankruptStageCd      9997
developer.bankruptStage.bankruptStageDesc    9997
developer.bankruptStage.bankruptLawUrl       9997
developer.orgBankruptMsgDttm                 9997
Length: 108, dtype: int64

In [4]:
df_dom = df_dom.loc[:, df_dom.isnull().mean() < .6]

In [39]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
df_dom.head(1).T

,0
id,13399
pdId,28141
region,72
address,"г Тюмень, ул Профсоюзная, д. 56"
floorMin,1
floorMax,1
objElemLivingCnt,0
objReady100PercDt,2024-12-31
wallMaterialShortDesc,Другое
objLkFinishTypeDesc,Без отделки


In [7]:
#убираю время из даты
df_dom.objTransferPlanDt = pd.to_datetime(df_dom.objTransferPlanDt).dt.date

In [8]:
df_dom.objTransferPlanDt

0        2025-02-28
1        2025-02-28
2        2022-08-31
3        2025-02-28
4        2022-08-30
            ...    
10074    2024-07-31
10075    2024-07-31
10076    2023-01-07
10077    2022-06-30
10078    2023-09-30
Name: objTransferPlanDt, Length: 10079, dtype: object

In [98]:
df = df_dom.groupby('objTransferPlanDt').agg({'objTransferPlanDt':'count'})
fig = px.line(df, x=df.index, y='objTransferPlanDt',color_discrete_sequence  = ['green'])
fig.update_layout(xaxis_title="Дата",yaxis_title="Кол-во домов", title='График выдачи ключей по всей России')
fig.update_traces(hoverinfo="all", hovertemplate="Дата: %{x}<br>Кол-во домов: %{y}")
fig.show()

In [124]:
df = df_dom.groupby('objLkFinishTypeDesc').agg({'id':'count'})

fig = px.bar(df, x=df.index, y='id',color_discrete_sequence  = ['orchid'])
fig.update_layout(xaxis_title="Тип отделки",yaxis_title="Кол-во", title='Типы отделки квартир')
fig.update_traces(hoverinfo="all", hovertemplate="Тип: %{x}<br>Кол-во: %{y}")
fig.show()

In [11]:
df = df_dom.groupby('objLkClassDesc').agg({'id':'count'})
fig = px.bar(df, x=df.index, y='id')
fig.update_layout(xaxis_title="Класс",yaxis_title="Кол-во", title='Класс квартир')
fig.update_traces(hoverinfo="all", hovertemplate="Класс: %{x}<br>Кол-во: %{y}")
fig.show()

In [12]:
#Самое дорогое жилье за кв.м.
df_dom.groupby('region').agg({'objPriceAvg':'mean'}).round().dropna().sort_values(by='objPriceAvg', ascending=False).head(5)

,objPriceAvg
region,
77,244589.0
78,155553.0
65,142215.0
25,116327.0
50,116326.0


In [13]:
#Самое дешевое жилье за кв.м.
df_dom.groupby('region').agg({'objPriceAvg':'mean'}).round().dropna().sort_values(by='objPriceAvg', ascending=True).head(5)

,objPriceAvg
region,
51,20442.0
5,25194.0
7,26918.0
20,30227.0
15,39715.0


In [14]:
#Самый дорогой кв.м
df_dom.sort_values(by='objPriceAvg', ascending = True).dropna().tail(1)[['region','objPriceAvg','address','objLkFinishTypeDesc','soldOutPerc','objLkClassDesc','generalContractorNm', 'developer.devShortCleanNm']].T

,7499
region,77
objPriceAvg,1730653.0
address,"г Москва, район Якиманка, переулок Большой Тол..."
objLkFinishTypeDesc,Без отделки
soldOutPerc,0.121931
objLkClassDesc,Элитный
generalContractorNm,ООО Смайнэкс Констракшн (ИНН: 7709492531)
developer.devShortCleanNm,СЗ ТОЛМАЧЕВСКИЙ


In [15]:
#Самый дешевый кв.м
df_dom.sort_values(by='objPriceAvg', ascending = True).dropna().head(1)[['region','objPriceAvg','address','objLkFinishTypeDesc','soldOutPerc','objLkClassDesc','generalContractorNm', 'developer.devShortCleanNm']].T

,1642
region,63
objPriceAvg,16826.0
address,"г Самара, район Ленинский, улица Галактионовск..."
objLkFinishTypeDesc,Без отделки
soldOutPerc,0.964121
objLkClassDesc,Элитный
generalContractorNm,ООО Бельведер (ИНН: 6311182876)
developer.devShortCleanNm,НОВОЕ ВРЕМЯ


In [17]:
#Больше всего домов строят в Краснодарском крае
df_dom.groupby('region').agg({'objLkClassDesc':'count'}).sort_values(by='objLkClassDesc', ascending = False).head(1)

,objLkClassDesc
region,
23,870


In [18]:
#Меньше всего строят в Мурманской области, Ингушетии, Магаданской области
df_dom.groupby('region').agg({'objLkClassDesc':'count'}).sort_values(by='objLkClassDesc', ascending = False).tail(3)

,objLkClassDesc
region,
51,1
6,1
49,1


In [31]:
def top_info(str):
    """Функция показывает статистику класса жилья по регионам или топ5 по классу жилья в регионе"""
    return df_dom.groupby(['region','objLkClassDesc']).agg({'id':'count'}).filter(like=str, axis=0).sort_values(by='id',ascending = False).head(5)
    

In [48]:
top_info('Бизнес')

,,id
region,objLkClassDesc,
77,Бизнес,271
23,Бизнес,82
78,Бизнес,78
50,Бизнес,67
38,Бизнес,39


In [36]:
top_info('63')

id
region objLkClassDesc    
63     Комфорт         89
       Типовой         65
       Элитный         11
       Бизнес           1

In [56]:
#Продано в процентах по регионам
df_dom.groupby('region').agg({'soldOutPerc':'mean'}).sort_values(by='soldOutPerc', ascending = False).head(3)

,soldOutPerc
region,
51,1.000000
8,0.921115
44,0.802724


In [77]:
# Макс кол-во этажей в доме
df_dom.loc[df_dom.floorMax.idxmax()][:10]

id                                                                   19622
pdId                                                                 47467
region                                                                  77
address                  г Москва, район Пресненский, набережная Красно...
floorMin                                                                76
floorMax                                                                76
objElemLivingCnt                                                       314
objReady100PercDt                                               2023-03-31
wallMaterialShortDesc                                       Монолит-кирпич
objLkFinishTypeDesc                                            Без отделки
Name: 1518, dtype: object

In [80]:
#Больше всего жилой площади в доме
df_dom.loc[df_dom.objFlatSq.idxmax()][:31]

id                                                                   37480
pdId                                                                365698
region                                                                  77
address                                                           г Москва
floorMin                                                                58
floorMax                                                                62
objElemLivingCnt                                                      1906
objReady100PercDt                                               2024-06-30
wallMaterialShortDesc                                               Другое
objLkFinishTypeDesc                                           Под чистовую
objLkFreePlanDesc                                                     Есть
objElemParkingCnt                                                     1020
objSquareLiving                                                   142978.7
rpdNum                   

In [81]:
#Больше всего квартир в доме
df_dom.loc[df_dom.objFlatCnt.idxmax()][:32]

id                                                                   45908
pdId                                                                530506
region                                                                  77
address                  г Москва, район Солнцево, ул Производственная,...
floorMin                                                                 2
floorMax                                                                25
objElemLivingCnt                                                      2676
objReady100PercDt                                               2024-09-30
wallMaterialShortDesc                                               Панель
objLkFinishTypeDesc                                            Без отделки
objLkFreePlanDesc                                                      Нет
objElemParkingCnt                                                      547
objSquareLiving                                                    82504.7
rpdNum                   

In [87]:
#Больше всего парковок в доме
df_dom.loc[df_dom.objElemParkingCnt.idxmax()][:20]

id                                                                   17375
pdId                                                                 25970
region                                                                  23
address                       г Краснодар, ул Заполярная, д. 39, литера 11
floorMin                                                                 5
floorMax                                                                 5
objElemLivingCnt                                                         0
objReady100PercDt                                               2026-06-30
wallMaterialShortDesc                                       Монолит-кирпич
objLkFinishTypeDesc                                           Под чистовую
objLkFreePlanDesc                                                      Нет
objElemParkingCnt                                                     3580
objSquareLiving                                                        0.0
rpdNum                   

In [13]:
#Популярная этажность домов
df_dom.groupby('floorMax').agg({'id':'count'}).sort_values(by='id', ascending= False).head(5)

,id
floorMax,
2,738
10,724
3,662
18,649
11,644


In [20]:
#Больше всего 2х этажных домов строят в новосибирской области
df_dom.groupby(['region','floorMax']).agg({'id':'count'}).query('floorMax ==2').sort_values(by='id',ascending = False).head(5)

,,id
region,floorMax,
54,2,161
23,2,115
50,2,91
72,2,53
78,2,37


In [30]:
# В разрезе класса домов
df_dom.groupby(['region','floorMax','objLkClassDesc']).agg({'id':'count'}).query('region==54 and floorMax == 2').sort_values(by='id',ascending = False).head(5)

id
region floorMax objLkClassDesc     
54     2        Типовой         144
                Комфорт          17

In [50]:
#Самые активные застройщики, занимаются строительством 2-3 этажных домов эконом класса 
df_dom.groupby(['developer.devShortCleanNm', 'region', 'floorMax','objLkClassDesc']).agg({'id': 'count','objPriceAvg': 'mean'}).round().sort_values(by = 'id', ascending=False).head(5)

,,,,id,objPriceAvg
developer.devShortCleanNm,region,floorMax,objLkClassDesc,,
АПРЕЛЕВКА С2,77,3,Типовой,115,42427.0
СЗ ГЕЛЕОН СТРОЙ,54,2,Типовой,72,NaN
СЗ ЮГА-СТРОЙ,54,2,Типовой,70,NaN
СЗ ГЕЛЕОН СТРОЙ,54,3,Комфорт,64,47907.0
СЗ КОШЕЛЕВ-ПРОЕКТ,40,3,Типовой,33,73389.0
